# Preliminary Design Calculations
---

**Importing libraries and notebook preparation**

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CP_HUMID_AIR, CoolPropWarning

Ignore warnings that come from module `hvac.fluids.fluid`:

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

Define a shortcut alias for instantiating `Quantity` objects:

In [3]:
Q_ = Quantity

## Outline

- Air supplied to a conditioned zone in a building must be cooled from 24 °C (RH 50%) to 13 °C. The mass flow rate of air is 1500 kg/hr.
- The refrigerant is R134a.
- The design outdoor air temperature is 35 °C, and the relative humidity of the outdoor air is 30 % RH.

We define the refrigerant:

In [4]:
R134a = Fluid('R134a')

We define the known air state at the entrance of the evaporator:

In [5]:
evp_air_in = HumidAir(Tdb=Q_(24, 'degC'), RH=Q_(50, 'pct'))

The absolute humidity ratio of this air is:

In [6]:
print(f"{evp_air_in.W.to('g / kg'):~P.3f}")

9.340 g/kg


We define the air state at the exit of the evaporator.

The air cooling process is, in general, both a sensible, and a latent air conditioning process. Moisture from the air will condense on the evaporator's cold external surface, which leads to dehumidification of the air. Assuming that the air leaving the evaporator is fully saturated (RH 100 %), we define the state of air leaving the evaporator as:

In [7]:
evp_air_out = HumidAir(Tdb=Q_(13, 'degC'), RH=Q_(100, 'pct'))

The absolute humidity ratio of this air is then:

In [8]:
print(f"{evp_air_out.W.to('g / kg'):~P.3f}")

9.372 g/kg


We notice that our assumption leads to air leaving the evaporator that contains more moisture than the air entering the evaporator, which means that the cooling process would add moisture instead of extracting it. As this is impossible, we need to alter our first assumption. If we assume that the relative humidity of the leaving air is 90 %, we get:

In [9]:
evp_air_out = HumidAir(Tdb=Q_(13, 'degC'), RH=Q_(90, 'pct'))
print(f"{evp_air_out.W.to('g / kg'):~P.3f}")

8.422 g/kg


We set the air mass flow rate through the evaporator:

In [10]:
evp_m_dot_air = Q_(1500, 'kg/hr')

## Heat Absorption Rate of the Refrigerant

The rate of heat that needs to be extracted from the air stream in the evaporator follows from:

In [11]:
Q_evp = evp_m_dot_air * (evp_air_in.h - evp_air_out.h)
print(f"{Q_evp.to('kW'):~P.3f}")

5.657 kW


## Selection of the Evaporation Temperature and Pressure

We select the target evaporation temperature of the refrigerant in the evaporator to be:

In [12]:
T_evp = Q_(10, 'degC')

We determine the corresponding evaporation pressure:

In [13]:
evp_rfg_sat_vap = R134a(T=T_evp, x=Q_(1.0, 'frac'))  # saturated vapor
P_evp = evp_rfg_sat_vap.P
print(f"evaporation pressure = {P_evp.to('bar'):~P.3f}")

evaporation pressure = 4.146 bar


## Selection of the Condensation Temperature and Pressure

We select the target condensing temperature of the refrigerant in the condenser to be:

In [14]:
T_cnd = Q_(50, 'degC')

We determine the corresponding condensing pressure:

In [15]:
cnd_rfg_sat_liq = R134a(T=T_cnd, x=Q_(0.0, 'frac'))  # saturated liquid
P_cnd = cnd_rfg_sat_liq.P
print(f"condensing pressure = {P_cnd.to('bar'):~P.3f}")

condensing pressure = 13.179 bar


## Mass flow rate of Refrigerant

The required mass flow rate of refrigerant can be determined once we've determined the state of refrigerant entering and leaving the evaporator. The refrigerant that leaves the evaporator needs to be superheated to a certain degree, which is a setting of the expansion device. The state of refrigerant entering the evaporator will depend on how much the refrigerant is subcooled when leaving the condenser.

We select the degree of superheating:

In [16]:
dT_sh = Q_(10, 'K')

We select the degree of subcooling:

In [17]:
dT_sc = Q_(5, 'K')

Now we can determine the state of refrigerant leaving the evaporator:

In [18]:
evp_rfg_out = R134a(T=T_evp.to('K') + dT_sh, P=P_evp)

To determine the state of refrigerant entering the evaporator, we can assume that the enthalpy of refrigerant entering the evaporator is about equal to the enthalpy of refrigerant leaving the condenser, as the expansion process can be considered to be isenthalpic. So, we first need to determine the state of refrigerant leaving the condenser:

In [19]:
cnd_rfg_out = R134a(T=T_cnd.to('K') - dT_sc, x=Q_(0.0, 'frac'))

Now we can determine the state of refrigerant entering the evaporator:

In [20]:
evp_rfg_in = R134a(h=cnd_rfg_out.h, P=P_evp)
print(f"vapor quality of entering refrigerant to evaporator = {evp_rfg_in.x.to('frac'):~P.2f}")

vapor quality of entering refrigerant to evaporator = 0.26 frac


Now that the states of refrigerant are determined on both sides of the evaporator, the mass flow rate of refrigerant, required to get the necessary heat absorption rate of the refrigerant in the evaporator, follows from:

In [21]:
m_dot_rfg = Q_evp / (evp_rfg_out.h - evp_rfg_in.h)
print(f"mass flow rate of refrigerant = {m_dot_rfg.to('kg/hr'):~P.3f}")

mass flow rate of refrigerant = 136.023 kg/h


## Heat Rejection Rate of the Refrigerant

The heat rate to be rejected in the condenser is the sum of the heat rate absorbed in the evaporator and the mechanical power delivered by the compressor to the refrigerant.

To determine the mechanical power of the compressor, a compressor selection software program is used (e.g. Coolselector2 from Danfoss). In this program, we need to enter:
- the required heat absorption rate in the evaporator,
- the selected evaporation temperature, 
- the selected degree of superheat,
- the selected condensing temperature, 
- the selected degree of subcooling, and 
- the type of refrigerant. 

The program will return appropriate compressor models to choose from.

When one of these compressors is selected, the program will show the performance characteristics of this compressor.

For this example, we've selected the compressor DANFOSS VTZ038-G. According to the selection program, the estimated discharge temperature of the refrigerant is about 85.4 °C, and the compressor power delivered to the refrigerant is 2.220 kW. With the indicated refrigerant discharge temperature, we can determine the state of refrigerant entering the condenser. We already know the state of refrigerant leaving the condenser, and we also know the mass flow rate of refrigerant. With this, we can determine the heat rate the refrigerant must reject to the air in the condenser.

We specify the discharge temperature of refrigerant leaving the compressor and entering the condenser:

In [22]:
T_dis = Q_(85.4, 'degC')

The state of refrigerant entering the condenser can now be determined:

In [23]:
cnd_rfg_in = R134a(T=T_dis, P=P_cnd)

The required heat rejection rate follows from:

In [24]:
Q_cnd = m_dot_rfg * (cnd_rfg_in.h - cnd_rfg_out.h)
print(f"heat rejection rate (based on discharge temperature) = {Q_cnd.to('kW'):~P.3f}")

heat rejection rate (based on discharge temperature) = 7.546 kW


## Mass Flow Rate of Air through the Condenser

The refrigerant enters the condenser at a temperature of around 85 °C and must leave the condenser at a temperature of around 45 °C, since we've selected a condensing temperature of 50 °C and the degree of subcooling to be 5 K. Air enters the condenser with a temperature of 35 °C. If we allow an air temperature rise of 10 K, we can determine the required mass flow rate of air as follows.

State of air entering the condenser:

In [25]:
cnd_T_air_in = Q_(35, 'degC')
cnd_air_in = HumidAir(Tdb=cnd_T_air_in, RH=Q_(30, 'pct'))

Temperature rise of air in the condenser:

In [26]:
cnd_dT_air = Q_(10, 'K')

State of air leaving the condenser:

In [27]:
cnd_T_air_out = cnd_T_air_in.to('K') + cnd_dT_air
cnd_air_out = HumidAir(Tdb=cnd_T_air_out, W=cnd_air_in.W)  # sensible heating process

Required mass flow rate of air through the condenser:

In [28]:
cnd_m_dot_air = Q_cnd / (CP_HUMID_AIR * (cnd_T_air_out - cnd_T_air_in))
print(f"mass flow rate of air through condenser = {cnd_m_dot_air.to('kg / hr'):~P.3f}")

mass flow rate of air through condenser = 2663.193 kg/h


## Frontal Area of the Evaporator

We select the air face velocity at the evaporator entrance:

In [29]:
evp_v_fa = Q_(2, 'm/s')

The air volume flow rate through the evaporator can now be determined as:

In [30]:
evp_V_dot_air = evp_m_dot_air / evp_air_in.rho
print(f"volume flow rate of air through evaporator = {evp_V_dot_air.to('m ** 3 / hr'):~P.3f}")

volume flow rate of air through evaporator = 1281.170 m³/h


The frontal area of the evaporator can now be determined as:

In [31]:
evp_A_fr = evp_V_dot_air / evp_v_fa
print(f"frontal area of evaporator = {evp_A_fr.to('m ** 2'):~P.4f}")

frontal area of evaporator = 0.1779 m²


To determine the width and the height of the frontal area, we select an aspect ratio (i.e., the ratio of height to width):

In [32]:
evp_AR = 1 / 3

The width of the frontal area then follows from:

In [33]:
evp_L1 = (evp_A_fr / evp_AR) ** 0.5
print(f"width of evaporator's frontal area = {evp_L1.to('mm'):~P.0f}")

width of evaporator's frontal area = 731 mm


The height of the frontal area:

In [34]:
evp_L3 = evp_A_fr / evp_L1
print(f"height of evaporator's frontal area = {evp_L3.to('mm'):~P.0f}")

height of evaporator's frontal area = 244 mm


## Frontal Area of the Condenser

We select the air face velocity at the condenser entrance:

In [35]:
cnd_v_fa = Q_(2, 'm/s')

The air volume flow rate through the condenser:

In [36]:
cnd_V_dot_air = cnd_m_dot_air / cnd_air_in.rho
print(f"volume flow rate of air through condenser = {cnd_V_dot_air.to('m ** 3 / hr'):~P.3f}")

volume flow rate of air through condenser = 2363.763 m³/h


The frontal area of the condenser:

In [37]:
cnd_A_fr = cnd_V_dot_air / cnd_v_fa
print(f"frontal area of condenser = {cnd_A_fr.to('m ** 2'):~P.4f}")

frontal area of condenser = 0.3283 m²


Aspect ratio of frontal area:

In [38]:
cnd_AR = 1 / 3

Width of the frontal area:

In [39]:
cnd_L1 = (cnd_A_fr / cnd_AR) ** 0.5
print(f"width of condenser's frontal area = {cnd_L1.to('mm'):~P.0f}")

width of condenser's frontal area = 992 mm


Height of the frontal area:

In [40]:
cnd_L3 = cnd_A_fr / cnd_L1
print(f"height of condenser's frontal area = {cnd_L3.to('mm'):~P.0f}")

height of condenser's frontal area = 331 mm


## Summary

**Evaporator**

In [41]:
print(
    f"entering refrigerant state: {evp_rfg_in.T.to('degC'):~P.2f}, {evp_rfg_in.x.to('frac'):~P.2f}",
    f"entering air state: {evp_air_in.Tdb.to('degC'):~P.2f}, RH {evp_air_in.RH.to('pct'):~P.2f}",
    f"air mass flow rate: {evp_m_dot_air.to('kg / hr'):~P.3f}",
    f"refrigerant mass flow rate: {m_dot_rfg.to('kg / hr'):~P.3f}",
    f"frontal area width = {evp_L1.to('mm'):~P.0f}",
    f"frontal area height = {evp_L3.to('mm'):~P.0f}",
    sep='\n'
)

entering refrigerant state: 10.00 °C, 0.26 frac
entering air state: 24.00 °C, RH 50.00 %
air mass flow rate: 1500.000 kg/h
refrigerant mass flow rate: 136.023 kg/h
frontal area width = 731 mm
frontal area height = 244 mm


**Condenser**

In [42]:
print(
    f"entering refrigerant state: {cnd_rfg_in.T.to('degC'):~P.2f}, {cnd_rfg_in.P.to('bar'):~P.3f}",
    f"entering air state: {cnd_air_in.Tdb.to('degC'):~P.2f}, {cnd_air_in.RH.to('pct'):~P.2f}",
    f"air mass flow rate: {cnd_m_dot_air.to('kg / hr'):~P.3f}",
    f"heat rejection rate: {Q_cnd.to('kW'):~P.3f}",
    f"frontal area width: {cnd_L1.to('mm'):~P.0f}",
    f"frontal area height: {cnd_L3.to('mm'):~P.0f}",
    sep='\n'
)

entering refrigerant state: 85.40 °C, 13.179 bar
entering air state: 35.00 °C, 30.00 %
air mass flow rate: 2663.193 kg/h
heat rejection rate: 7.546 kW
frontal area width: 992 mm
frontal area height: 331 mm
